In [7]:
import torch
import json
from torch.utils.data import DataLoader, IterableDataset
import torch.utils.data as data
from abc import ABC, abstractmethod
import cv2
import os
from PIL import Image

### Carga de datos

In [49]:
import torch
import json
from torch.utils.data import DataLoader, IterableDataset
import torch.utils.data as data
import cv2
import os
from glob import glob
import json

LIMIT = 5000
batch_size = 15

from torch.utils.data import Dataset

class baseDataset(Dataset):
    def __init__(self, json_file):
        self.labels=[]
        self.limit = LIMIT
        self.img_size = (0,0)
        self.images = [] #Aqui guardamos todas las imagenes del dataset
        self.json_file = json_file
        
        self.load_data()
        print("Se han cargado {} imagenes".format(self.__len__()))
    
    #Devuelve la imagen de la posicion index de la lista de imagenes a usar
    def __getitem__(self,index):
        img = self.images[index] 
        img_labels = self.labels[index]
        return img, img_labels
    
    def __len__(self):
        return len(self.images)

    
    def load_data(self):        
        index = 0
        json_data = json.loads(open(self.json_file, "rb").read())
        for data in json_data:
            img_name = data['imagen']
            point = data['punto']   # con estos datos se crean las etiquetas y se añaden a self.labels
            label = [img_name, point] # creamos las etiquetas de las imagenes y las añadimos a la lista de etiquetas
            self.labels.append(label)
            img_path = 'VueltaRuido_condiciones/out/' + img_name
            
            if(os.path.isfile((img_path))):   
                print(img_path)
                
                img_aux = Image.open(img_path)
                width = img_aux.width/2
                height = img_aux.height/2
                mode = img_aux.mode      
                print("Width: {} Height: {} Mode: {} ".format(width, height, mode))
                
                self.img_size = int(width), int(height)
                
                
                
                image = cv2.imread(img_path, 0)
                image = cv2.resize(image, self.img_size, interpolation=cv2.INTER_AREA)
                image = torch.from_numpy(image).type(torch.FloatTensor)
                image = (image/255.)*2.-1.
                self.images.append(image)

            if index % 100 == 0:
                print(index)
            index += 1
            
            if index > self.limit:
                print("Stop loading data, limit reached")
                break


dataset = baseDataset('VueltaRuido_condiciones/ConRuido_cond_clim.json')
trainloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=1)



VueltaRuido_condiciones/out/0.png
Width: 400.0 Height: 300.0 Mode: RGB 
0
VueltaRuido_condiciones/out/1.png
Width: 400.0 Height: 300.0 Mode: RGB 
VueltaRuido_condiciones/out/2.png
Width: 400.0 Height: 300.0 Mode: RGB 
VueltaRuido_condiciones/out/3.png
Width: 400.0 Height: 300.0 Mode: RGB 
VueltaRuido_condiciones/out/10.png
Width: 400.0 Height: 300.0 Mode: RGB 
VueltaRuido_condiciones/out/11.png
Width: 400.0 Height: 300.0 Mode: RGB 
VueltaRuido_condiciones/out/12.png
Width: 400.0 Height: 300.0 Mode: RGB 
VueltaRuido_condiciones/out/13.png
Width: 400.0 Height: 300.0 Mode: RGB 
VueltaRuido_condiciones/out/14.png
Width: 400.0 Height: 300.0 Mode: RGB 
VueltaRuido_condiciones/out/15.png
Width: 400.0 Height: 300.0 Mode: RGB 
VueltaRuido_condiciones/out/16.png
Width: 400.0 Height: 300.0 Mode: RGB 
VueltaRuido_condiciones/out/17.png
Width: 400.0 Height: 300.0 Mode: RGB 
VueltaRuido_condiciones/out/18.png
Width: 400.0 Height: 300.0 Mode: RGB 
VueltaRuido_condiciones/out/19.png
Width: 400.0 Heigh

VueltaRuido_condiciones/out/205.png
Width: 400.0 Height: 300.0 Mode: RGB 
VueltaRuido_condiciones/out/206.png
Width: 400.0 Height: 300.0 Mode: RGB 
VueltaRuido_condiciones/out/207.png
Width: 400.0 Height: 300.0 Mode: RGB 
VueltaRuido_condiciones/out/208.png
Width: 400.0 Height: 300.0 Mode: RGB 
VueltaRuido_condiciones/out/209.png
Width: 400.0 Height: 300.0 Mode: RGB 
VueltaRuido_condiciones/out/210.png
Width: 400.0 Height: 300.0 Mode: RGB 
VueltaRuido_condiciones/out/211.png
Width: 400.0 Height: 300.0 Mode: RGB 
VueltaRuido_condiciones/out/212.png
Width: 400.0 Height: 300.0 Mode: RGB 
VueltaRuido_condiciones/out/213.png
Width: 400.0 Height: 300.0 Mode: RGB 
VueltaRuido_condiciones/out/214.png
Width: 400.0 Height: 300.0 Mode: RGB 
VueltaRuido_condiciones/out/215.png
Width: 400.0 Height: 300.0 Mode: RGB 
VueltaRuido_condiciones/out/216.png
Width: 400.0 Height: 300.0 Mode: RGB 
VueltaRuido_condiciones/out/217.png
Width: 400.0 Height: 300.0 Mode: RGB 
VueltaRuido_condiciones/out/218.png
Wi

In [50]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

### Creacion de la red

In [51]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [52]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

TypeError: cross_entropy_loss(): argument 'target' (position 2) must be Tensor, not list